In [11]:
import logging
from cimpy import *
from cimpy.cgmes_v2_4_15.Base import short_profile_name
import os
import glob
import pytest_check as check
import pickle
from pathlib import Path

from cimpy import utils
from cimpy.cgmes_v2_4_15 import BaseVoltage

logging.basicConfig(filename='Test_export_with_exported_files.log',
                    level=logging.INFO, filemode='w')

res = {'meta_info': {
  'namespaces': {
    "cim":   "http://iec.ch/TC57/2013/CIM-schema-cim16#",
    "entsoe":"http://entsoe.eu/CIM/SchemaExtension/3/1#",
    "md":    "http://iec.ch/TC57/61970-552/ModelDescription/1#",
    "rdf":   "http://www.w3.org/1999/02/22-rdf-syntax-ns#"},
  'urls': {}, 
  'author': 'DPsim'},
  'topology': {}
}

B1 = BaseVoltage(mRID="BaseVoltage", nominalVoltage=100)
res["topology"]["BaseVoltage"] = B1
res = utils.add_TopologicalNode(import_result=res, version="cgmes_v2_4_15", BaseVoltage=B1, v=100, angle=0, mRID = "Node1")
res = utils.add_TopologicalNode(import_result=res, version="cgmes_v2_4_15", BaseVoltage=B1, v=100, angle=0, mRID = "Node2")
res = utils.add_ACLineSegment(import_result=res, version="cgmes_v2_4_15", mRID_1="Node1", mRID_2="Node2", r=0.1, x=0.2, bch=0, gch=0, BaseVoltage=B1, name="Line1")
res = utils.add_EnergyConsumer(res, "cgmes_v2_4_15","Node1", 100, 100, BaseVoltage= B1, name = "EC1")
res = utils.add_EnergyConsumer(res, "cgmes_v2_4_15","Node2", 100, 100, BaseVoltage= B1, name = "EC2")

In [12]:
# CIM to graph
import pygraphviz as pgv

#       
topology = res['topology']

#
nodes_list = []
terminals_list = []
lineSegment_list = []       # two terminal components
energyConsumer_list = []  # one terminal components

for mRID in topology.keys():
    class_name = topology[mRID].__class__.__name__
    if class_name == "Terminal":
        terminals_list.append(mRID)
    elif class_name == 'TopologicalNode':
        nodes_list.append(mRID)
    elif class_name in ['EnergyConsumer']:
        energyConsumer_list.append(mRID)
    elif class_name == 'ACLineSegment':
        lineSegment_list.append(mRID)

# create gundirected raph
#g = graphviz.Graph('G', filename='hello.gv')
G = pgv.AGraph()
G.graph_attr["splines"] = "polyline"
G.graph_attr["layout"] = "neato"
G.node_attr["shape"] = "circle"
G.edge_attr["color"] = "red"

# add nodes
for node in nodes_list:
    G.add_node(node, shape="point")

# add components
for comp in energyConsumer_list:
    #G.add_node(comp, shape="plaintext")
    G.add_node(comp, shape="point")
    for terminal in topology[comp].Terminals:
        G.add_edge(comp, terminal.TopologicalNode.mRID)

for comp in lineSegment_list:
    #G.add_node(comp, shape="rectangle", height=0.05)
    #G.add_node(comp, shape="plaintext")
    #G.add_node(comp, shape="point")
    G.add_node(comp, shape="none")
    for terminal in topology[comp].Terminals:
        G.add_edge(comp, terminal.TopologicalNode.mRID)

# generate a layout--this creates `pos` attributes for both nodes and edges
G.layout()
G.draw("file.png")

In [13]:
import cimpy.cgmes_v2_4_15.Diagram as Diagram
import cimpy.cgmes_v2_4_15.OrientationKind as OrientationKind
import uuid

# add diagram object
topology=res['topology']
uuid_diagram=str(uuid.uuid4())

#
topology[uuid_diagram] = Diagram(mRID=uuid_diagram, name="Test Diagram", orientation=OrientationKind())


In [14]:
import cimpy.cgmes_v2_4_15.DiagramObject as DiagramObject
import cimpy.cgmes_v2_4_15.DiagramObjectPoint as DiagramObjectPoint 

# add nodes to diagram
for node in nodes_list:
    n = G.get_node(node)
    x_pos = float(n.attr["pos"].split(",")[0])
    y_pos = float(n.attr["pos"].split(",")[1])
    uuid_diagObject=str(uuid.uuid4())
    topology[uuid_diagObject]=DiagramObject(mRID=uuid_diagObject, Diagram=topology[uuid_diagram], IdentifiedObject=topology[node])
    uuid_diagObjectPoint1=str(uuid.uuid4())
    topology[uuid_diagObjectPoint1]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=1, xPosition=x_pos-15, yPosition=y_pos)
    uuid_diagObjectPoint2=str(uuid.uuid4())
    topology[uuid_diagObjectPoint2]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=2, xPosition=x_pos+15, yPosition=y_pos)
    
## add energy consumers to diagram
for component in energyConsumer_list:
    n = G.get_node(component)
    x_pos = float(n.attr["pos"].split(",")[0])
    y_pos = float(n.attr["pos"].split(",")[1])
    uuid_diagObject=str(uuid.uuid4())
    topology[uuid_diagObject]=DiagramObject(mRID=uuid_diagObject, Diagram=topology[uuid_diagram], IdentifiedObject=topology[component])
    uuid_diagObjectPoint1=str(uuid.uuid4())
    topology[uuid_diagObjectPoint1]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=1, xPosition=x_pos, yPosition=y_pos)
    
    # add terminal
    for terminal in topology[component].Terminals:
        n = G.get_node(component)
        uuid_diagObject=str(uuid.uuid4())
        topology[uuid_diagObject]=DiagramObject(mRID=uuid_diagObject, Diagram=topology[uuid_diagram], IdentifiedObject=topology[terminal.mRID])
        uuid_diagObjectPoint1=str(uuid.uuid4())
        topology[uuid_diagObjectPoint1]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=1, xPosition=x_pos, yPosition=y_pos)
        uuid_diagObjectPoint2=str(uuid.uuid4())
        topology[uuid_diagObjectPoint2]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=2, xPosition=x_pos, yPosition=y_pos-10)
        
        # draw connection to node
        n2 = G.get_node(topology[component].Terminals[0].TopologicalNode.mRID)
        x_pos = float(n2.attr["pos"].split(",")[0])
        y_pos = float(n2.attr["pos"].split(",")[1])
        uuid_diagObjectPoint3=str(uuid.uuid4())
        topology[uuid_diagObjectPoint3]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=3, xPosition=x_pos, yPosition=y_pos)
        # edges
        #edge = G.get_edge(component, terminal.TopologicalNode.mRID)
        #pos = edge.attr["pos"].split(" ")
        #for idx, value in enumerate(pos):
        #    x_pos = value.split(",")[0]
        #    y_pos = value.split(",")[1]
        
## add ACLineSegments to diagram
for component in lineSegment_list:
    uuid_diagObject=str(uuid.uuid4())
    topology[uuid_diagObject]=DiagramObject(mRID=uuid_diagObject, Diagram=topology[uuid_diagram], IdentifiedObject=topology[component])
    
    #draw connection to first node
    n1 = G.get_node(topology[component].Terminals[0].TopologicalNode.mRID)
    x_pos = float(n1.attr["pos"].split(",")[0])
    y_pos = float(n1.attr["pos"].split(",")[1])
    uuid_diagObjectPoint_n1=str(uuid.uuid4())
    topology[uuid_diagObjectPoint_n1]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=1, xPosition=x_pos, yPosition=y_pos)

    # draw line    
    n = G.get_node(component)
    x_pos = float(n.attr["pos"].split(",")[0])
    y_pos = float(n.attr["pos"].split(",")[1])
    uuid_diagObjectPoint1=str(uuid.uuid4())
    topology[uuid_diagObjectPoint1]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=2, xPosition=x_pos, yPosition=y_pos-10)
    uuid_diagObjectPoint2=str(uuid.uuid4())
    topology[uuid_diagObjectPoint2]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=3, xPosition=x_pos, yPosition=y_pos+10)
    
    #draw connection to first node
    n2 = G.get_node(topology[component].Terminals[1].TopologicalNode.mRID)
    x_pos = float(n2.attr["pos"].split(",")[0])
    y_pos = float(n2.attr["pos"].split(",")[1])
    uuid_diagObjectPoint_n2=str(uuid.uuid4())
    topology[uuid_diagObjectPoint_n2]=DiagramObjectPoint(DiagramObject=topology[uuid_diagObject], sequenceNumber=4, xPosition=x_pos, yPosition=y_pos)
        


In [15]:
dir(G)
print(G.to_string())
edge = G.get_edge("EC1", "Node1")
pos = edge.attr["pos"].split(" ")
x_pos = pos[0].split(",")[0]
y_pos = pos[0].split(",")[1]
print(pos)
print(x_pos)
print(y_pos)

strict graph "" {
	graph [bb="0,0,124.97,263.32",
		layout=neato,
		splines=polyline
	];
	node [label="\N",
		shape=circle
	];
	edge [color=red];
	Node1	[height=0.05,
		pos="25.78,69.987",
		shape=point,
		width=0.05];
	Node2	[height=0.05,
		pos="103.01,192.06",
		shape=point,
		width=0.05];
	EC1	[height=0.05,
		pos="1.8,1.8",
		shape=point,
		width=0.05];
	EC1 -- Node1	[pos="2.4918,3.7672 5.9935,13.724 21.73,58.472 25.131,68.141"];
	EC2	[height=0.05,
		pos="123.17,261.52",
		shape=point,
		width=0.05];
	EC2 -- Node2	[pos="122.59,259.51 119.65,249.37 106.41,203.79 103.55,193.94"];
	Line1	[height=0.5,
		pos="66.367,129.79",
		shape=none,
		width=0.75];
	Line1 -- Node1	[pos="53.969,111.52 43.898,96.683 30.754,77.315 26.892,71.625"];
	Line1 -- Node2	[pos="76.965,147.8 86.095,163.32 98.357,184.16 101.97,190.3"];
}

['2.4918,3.7672', '5.9935,13.724', '21.73,58.472', '25.131,68.141']
2.4918
3.7672


In [16]:
res['topology'] = topology
activeProfileList = ['SV', 'EQ', 'DL', 'TP']
cimpy.cim_export(res, 'test', 'cgmes_v2_4_15', activeProfileList)

[ERROR:] File test_StateVariables.xml already exists. Delete file or change file name to serialize CGMES classes.


NameError: name 'exit' is not defined